In [3]:
"""import blockulib.models as blom
import torch

model = blom.ConvModel()
input_tensor = torch.randn(1, 1, 9, 9)
output = model(input_tensor)
print("New model sample output: ", output)

PATH = "models/conv_model.pth"
torch.save(model.state_dict(), PATH) """

'import blockulib.models as blom\nimport torch\n\nmodel = blom.ConvModel()\ninput_tensor = torch.randn(1, 1, 9, 9)\noutput = model(input_tensor)\nprint("New model sample output: ", output)\n\nPATH = "models/conv_model.pth"\ntorch.save(model.state_dict(), PATH) '

In [4]:
import blockulib
import blockulib.models as blom
import torch

class PlayingLoop():
    
    def __init__(self, model_path = "models/conv_model.pth", architecture = blom.ConvModel):
        self.model = architecture()
        state_dict = torch.load(model_path)
        self.model.load_state_dict(state_dict)
        self.generator = blockulib.BlockGenerator()
        
    def __call__(self, num_games = 1, batch_size = 128):
        pos_list = [[torch.zeros(9, 9)] for i in range(num_games)]
        state = [True for i in range(num_games)]
        active_games = num_games
        move = 0
        
        while (active_games > 0):
            move += 1
            if (move == 1 or move%5 == 0):
                print("Entering move phase: ", move)
            new_index = []
            for i in range(num_games):
                if state[i]:
                    new_index.append(i)
            boards = [pos_list[new_index[i]][-1].clone() for i in range(active_games)]
            pos, ind = blockulib.possible_moves(boards, self.generator)
            
            state = [False for i in range(num_games)]
            active_games = 0
            if (move == 1 or move%5 == 0):
                print("Possible moves count: ", ind.shape[0])
            for i in range(ind.shape[0]):
                index = new_index[int(ind[i].item())]
                if (not state[index]):
                    state[index] = True
                    pos_list[index].append(pos[i])
                    active_games +=1
        
        print("ended after ", move, " moves")            
        return pos_list

In [5]:
play = PlayingLoop()

In [6]:
import time
start_time = time.time()

wynik = play(num_games = 10)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Entering move phase:  1
Possible moves count:  59299
Entering move phase:  5
Possible moves count:  43482
Entering move phase:  10
Possible moves count:  29734
Entering move phase:  15
Possible moves count:  18821
Entering move phase:  20
Possible moves count:  11000
Entering move phase:  25
Possible moves count:  5368
Entering move phase:  30
Possible moves count:  2807
Entering move phase:  35
Possible moves count:  1499
Entering move phase:  40
Possible moves count:  512
Entering move phase:  45
Possible moves count:  384
Entering move phase:  50
Possible moves count:  180
Entering move phase:  55
Possible moves count:  102
Entering move phase:  60
Possible moves count:  15
Entering move phase:  65
Possible moves count:  6
Entering move phase:  70
Possible moves count:  9
Entering move phase:  75
Possible moves count:  11
Entering move phase:  80
Possible moves count:  6
Entering move phase:  85
Possible moves count:  9
Entering move phase:  90
Possible moves count:  14
Entering mov

In [7]:
print(len(wynik))

1000


In [8]:
cases = 0
for i in range(1000):
    cases += len(wynik[i])
print(cases, "cases int total")

26492 cases int toal
